In [2]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import torchvision.transforms as transforms
import torch.nn.functional as F

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
import cv2
import face_recognition
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture(r'C:\Users\ahmad\Downloads\853889-hd_1920_1080_25fps.mp4')

# Initialize lists to store known face encodings and their corresponding IDs
known_face_encodings = []
known_face_ids = []

# Function to assign unique ID to each new person
def get_unique_id():
    if len(known_face_ids) == 0:
        return 0
    else:
        return max(known_face_ids) + 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Check if the frame is empty or invalid
    if frame is None or frame.size == 0:
        print("Warning: Empty or invalid frame captured.")
        continue

    # Convert the frame to RGB (face_recognition expects RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    updated_face_ids = []

    # Loop over each detected face
    for i, face_encoding in enumerate(face_encodings):
        match_found = False

        # Compare this face's encoding with the known faces
        for known_encoding, face_id in zip(known_face_encodings, known_face_ids):
            match = face_recognition.compare_faces([known_encoding], face_encoding, tolerance=0.6)
            if match[0]:
                updated_face_ids.append((face_locations[i], face_id))
                match_found = True
                break

        # If no match found, assign a new ID
        if not match_found:
            new_id = get_unique_id()  # Get a new unique ID
            known_face_encodings.append(face_encoding)
            known_face_ids.append(new_id)
            updated_face_ids.append((face_locations[i], new_id))

    # Draw bounding boxes and IDs for each face
    for (top, right, bottom, left), face_id in updated_face_ids:
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {face_id}', (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with bounding boxes and IDs
    cv2.imshow('Video', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.